In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [2]:
ap = PlainTextAuthProvider(username='cassandra', password='password123')

cluster = Cluster(auth_provider=ap)
session = cluster.connect()

In [3]:
session.execute("""
    create keyspace if not exists default 
    with replication = {
        'class' : 'NetworkTopologyStrategy',
        'datacenter1' : 1
    };
""");

In [4]:
session.set_keyspace('default')

## Створюємо тестову таблицю

In [5]:
session.execute("""
    create table if not exists "messages"
    (
        id          int,
        body        text,
        PRIMARY KEY (id)
    );
""");

## Пишемо тестові дані в базу 

In [6]:
for identifier in range(10):
    session.execute("""
        insert into messages (id, body) values (%s, %s);
    """, (identifier, f"Message {identifier}"))

## Для кожного рядка виведемо ноду, на якій він зберігається  

In [7]:
for row in session.execute('select * from messages;'):
    !docker compose exec cassandra-node1 nodetool -h localhost getendpoints default messages {row.id}    

192.168.80.3
192.168.80.3
192.168.80.2
192.168.80.3
192.168.80.2
192.168.80.3
192.168.80.3
192.168.80.2
192.168.80.3
192.168.80.3


Бачимо, що рядки записалися на різні ноди.